In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import db_helper
import datetime
import networkx as nx
from node2vec import Node2Vec
from datetime import datetime
from gensim.models import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import importlib
import graph_constructor as GC
import n2v as N2V
import utils
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [30]:
import gc
gc.collect()

0

In [43]:
PROJECT_PATH='/home/hmenn/Workspace/CSE496_Node2Vec_2018'
MONTH_TAG_EN="month"
YEAR_TAG_EN="year"
DAY_TAG_EN="day"

MONTH_NAMES_EN = {1:"JAN",2:"FEB",3:"MAR",4:"APR",
          5:"MAY",6:"JUN",7:"JUL",8:"AUG",
         9:"SEP",10:"OCT",11:"NOV",12:"DEC"}

MONTH_TAG_TR="AY"
YEAR_TAG_TR="YIL"
DAY_TAG_TR="GUN"

MONTH_NAMES_TR = {1:"OCA",2:"SUB",3:"MAR",4:"NIS",
          5:"MAY",6:"HAZ",7:"TEM",8:"AGU",
         9:"EYL",10:"EKI",11:"KAS",12:"ARA"}

In [28]:
db = db_helper.DBHelper()
db.connectMysql(user="root",passwd="Hasan5695*",db="SocialMediaDB")
tweets = db.getTweetsFromMysql("TweetsShort")
db_helper.saveTweetsAsJson(PROJECT_PATH+'/dataset/SocialMediaDataset/tweets.json', tweets)

In [83]:
tweets = db_helper.getTweetsFromJson(PROJECT_PATH+'/dataset/SocialMediaDataset/tweets.json')

In [84]:
print(tweets[:5])

[############ Tweet: ############ 
SendByScreenName:cakrr_ 
UserID:u1623755515 
TweetID:t763141405766250496 
InReplyToStatusID:t763141168771244032 
InReplyToUserID:u757176735158923264 
Date:2016-08-10 01:34:24 
Text:@Wesleyoloji10 OGLAK HAZİNE ARIYOR AQ:das:asd 
, ############ Tweet: ############ 
SendByScreenName:selcukarici 
UserID:u154236788 
TweetID:t763577912696135680 
InReplyToStatusID:t-1 
InReplyToUserID:u-1 
Date:2016-08-11 06:28:55 
Text:Sn.CB'nımızın çağrısıyla 3 banka konut kredisi faiz oranlarını ℅1'in altına çekti..Bu olumlu bir adım.

Ancak şişen piyasa nasıl düşecek? 
, ############ Tweet: ############ 
SendByScreenName:selcukarici 
UserID:u154236788 
TweetID:t763578688487251969 
InReplyToStatusID:t-1 
InReplyToUserID:u-1 
Date:2016-08-11 06:32:00 
Text:Konut kira/satışları düşünüldüğünde piyasa öylesine şişti ki.Örneğin İstanbul'da 1000 liranın altında ev kiralamanız nerdeyse mümkün değil! 
, ############ Tweet: ############ 
SendByScreenName:aa_finans 
UserID:u5476251

In [105]:
def create_all_graph(tweets):
    G1 = nx.DiGraph()
    G1.add_node(YEAR_TAG_TR)
    G1.add_node(MONTH_TAG_TR)
    G1.add_node(DAY_TAG_TR)

    prevYear = None
    prevMonth = None
    prevDay = None

    prevYear = tweets[0].date.year
    prevMonth = tweets[0].date.month
    prevDay = tweets[0].date.day

    for tweet in tweets:
        G1.add_edge(YEAR_TAG_TR,str(tweet.date.year),weight=1.0)
        
        if(prevYear < tweet.date.year):
            G1.add_edge(str(prevYear),str(tweet.date.year),weight=1.0)
            prevYear = tweet.date.year
            prevMonth = tweet.date.month
            prevDay = tweet.date.day
          
        monthFormat = "{month}.{year}".format(month=MONTH_NAMES_TR[tweet.date.month],year=tweet.date.year)
        G1.add_edge(str(tweet.date.year),monthFormat,weight=1.0)
        
        if(prevMonth < tweet.date.month):
            prevMonthFormat = "{month}.{year}".format(month=MONTH_NAMES_TR[prevMonth],year=tweet.date.year)
            G1.add_edge(prevMonthFormat,monthFormat,weight=1.0)
            prevMonth = tweet.date.month
            prevDay = tweet.date.day
        
        #G1.add_edge(MONTH_TAG,monthFormat,weight=1.0)
        fullDateFormat = "{day}.{month}.{year}".format(day=tweet.date.day,month=tweet.date.month,year=tweet.date.year)
        if prevDay < tweet.date.day:
            prevFullDateFormat = "{day}.{month}.{year}".format(day=prevDay,month=tweet.date.month,year=tweet.date.year)
            G1.add_edge(prevFullDateFormat,fullDateFormat,weight=1.0)
            prevDay = tweet.date.day
            
        G1.add_edge(tweet.tweetID,fullDateFormat,weight=1.0)
        G1.add_edge(tweet.userID, tweet.tweetID,weight=1.0)
        
        # if tweet replied and db has actuaal(replied tweet, add to graph)
        # cevap verilen tweet db de ise, o tweete baglanti ekle
        for i in tweets:
            if i.tweetID == tweet.inReplyToStatusID:
                G1.add_edge(tweet.tweetID, tweet.inReplyToStatusID,weight=1.0)
        """
            # if tweet is a reply and not replied for own tweet, 
            if tweet.inReplyToUserID != tweet.userID:
                G1.add_edge(tweet.tweetID, tweet.inReplyToStatusID)
                #G1.add_edge(tweet.inReplyToStatusID, tweet.tweetID)
        """
        
        G1.add_edge(monthFormat,fullDateFormat,weight=1.0)
    return G1

In [106]:
startTime = datetime.now()
graph = create_all_graph(tweets[:1000])
endTime = datetime.now()
print("Graph construct time: ",endTime-startTime)
print("Number of edges: ",graph.number_of_edges())
print("Number of nodes: ",graph.number_of_nodes())

Graph construct time:  0:00:00.125735
Number of edges:  2069
Number of nodes:  1092


In [101]:
graph.edges()#["YIL"]

OutEdgeView([('YIL', '2016'), ('2016', 'AGU.2016'), ('AGU.2016', '10.8.2016'), ('AGU.2016', '11.8.2016'), ('AGU.2016', '12.8.2016'), ('AGU.2016', '13.8.2016'), ('AGU.2016', '14.8.2016'), ('AGU.2016', '15.8.2016'), ('AGU.2016', '16.8.2016'), ('AGU.2016', '17.8.2016'), ('AGU.2016', '18.8.2016'), ('AGU.2016', '19.8.2016'), ('AGU.2016', '20.8.2016'), ('AGU.2016', '22.8.2016'), ('AGU.2016', '23.8.2016'), ('AGU.2016', '24.8.2016'), ('t763141405766250496', '10.8.2016'), ('10.8.2016', '11.8.2016'), ('u1623755515', 't763141405766250496'), ('11.8.2016', '12.8.2016'), ('t763577912696135680', '11.8.2016'), ('u154236788', 't763577912696135680'), ('u154236788', 't763578688487251969'), ('u154236788', 't766883912891138048'), ('u154236788', 't766887626460856320'), ('u154236788', 't766912872039456768'), ('t763578688487251969', '11.8.2016'), ('t763685474959495168', '11.8.2016'), ('u547625189', 't763685474959495168'), ('u547625189', 't765099162794942464'), ('u547625189', 't765828355077931008'), ('u5476251

In [107]:
graphN2V = N2V.Graph(graph, True, 1.0, 1.0)
graphN2V.preprocess_transition_probs()
walks = graphN2V.simulate_walks(5000, 80)

Walk iteration:
500 / 5000
1000 / 5000
1500 / 5000
2000 / 5000
2500 / 5000
3000 / 5000
3500 / 5000
4000 / 5000
4500 / 5000
5000 / 5000


In [108]:
model = N2V.learn_embeddings(walks,"graphAll")

In [110]:
column_names = ['2016', 'AGU.2016', '24.8.2016', "20.8.2016","t766739153195266049","t766738735786491905", "t766744289342808069"]
matrix1 = np.stack((model.wv[column_names]))
cosines1 = cosine_similarity(matrix1)
df1 = pd.DataFrame(cosines1, columns=column_names, index=column_names)
df1

,2016,AGU.2016,24.8.2016,20.8.2016,t766739153195266049,t766738735786491905,t766744289342808069
2016,1.000000,0.618084,0.092613,0.129632,0.196958,0.221195,0.250900
AGU.2016,0.618084,1.000000,0.070128,0.106470,0.221856,0.153677,0.219371
24.8.2016,0.092613,0.070128,1.000000,0.193325,0.144194,0.165400,0.199692
20.8.2016,0.129632,0.106470,0.193325,1.000000,0.081073,0.089201,0.007137
t766739153195266049,0.196958,0.221856,0.144194,0.081073,1.000000,0.688404,0.353447
t766738735786491905,0.221195,0.153677,0.165400,0.089201,0.688404,1.000000,0.351324
t766744289342808069,0.250900,0.219371,0.199692,0.007137,0.353447,0.351324,1.000000
